## **Installing and Importing necessary libraries**

In [ ]:
!pip install -U sentence-transformers
!pip install rake-nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import seaborn as sns
import re
import string
from string import punctuation
import nltk
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import tokenize
from operator import itemgetter
import math
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))
from rake_nltk import Rake
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from numpy import unique

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## **Reading the Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/drugsComTest_raw.csv")

## **Pre-processing the data**

### **Removing symbols**

In [ ]:
df=pd.DataFrame(data=(data["drugName"],data["review"]))
df=df.T
for i in df.index:
  df['review'][i]=df['review'][i].replace("&#039;","'")
  df['review'][i]=df['review'][i].replace(",","")
  df['review'][i]=df['review'][i].replace("\\","")
  df['review'][i]=df['review'][i].replace("\"","")
  df['review'][i]=df['review'][i].replace("\"]","")
  df['review'][i]=df['review'][i].replace("[\"","")
  df['review'][i]=df['review'][i].replace("!",".")
  df['review'][i]=df['review'][i].replace("?",".")
df

,drugName,review
0,Mirtazapine,I've tried a few antidepressants over the year...
1,Mesalamine,My son has Crohn's disease and has done very w...
2,Bactrim,Quick reduction of symptoms
3,Contrave,Contrave combines drugs that were used for alc...
4,Cyclafem 1 / 35,I have been on this birth control for one cycl...
...,...,...
53761,Tamoxifen,I have taken Tamoxifen for 5 years. Side effec...
53762,Escitalopram,I've been taking Lexapro (escitaploprgram) sin...
53763,Levonorgestrel,I'm married 34 years old and I have no kids. T...
53764,Tapentadol,I was prescribed Nucynta for severe neck/shoul...


### **Grouping the data by medicine names**

In [ ]:
df2 = df.groupby("drugName") 
df_new = df2["review"].apply(list) 
df_new = df_new.reset_index() 
df_new

,drugName,review
0,A / B Otic,[These drops helped me so much. I was in sever...
1,Abacavir / dolutegravir / lamivudine,[I discovered my new status in July 2017 VL 94...
2,Abacavir / lamivudine,[I've used this since first diagnosed and star...
3,Abatacept,[In UK had this on NHS for last 7 months. Impr...
4,Abilify,[After having to pay a but load out of pocket ...
...,...,...
2632,ZzzQuil,[Gave me rapid heart beats .........]
2633,depo-subQ provera 104,[Just recently have got Depo subQ provera for ...
2634,ella,[Hello strangers. Im here to share my experien...
2635,femhrt,[Hot flashes completely gone in less than 2 we...


### **Stopword Removal**

In [ ]:
def get_text_processing(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

### **Creating a separate column for the refined text**

In [ ]:
df_new['rev'] = df_new['review'].apply(get_text_processing)
df_new.head()

,drugName,review,rev
0,A / B Otic,[These drops helped me so much. I was in sever...,drops helped much. severe pain 8 days finally ...
1,Abacavir / dolutegravir / lamivudine,[I discovered my new status in July 2017 VL 94...,discovered new status July 2017 VL 94K CD4 126...
2,Abacavir / lamivudine,[I've used this since first diagnosed and star...,I've used since first diagnosed starting HAART...
3,Abatacept,[In UK had this on NHS for last 7 months. Impr...,UK NHS last 7 months. Improved within first mo...
4,Abilify,[After having to pay a but load out of pocket ...,pay load pocket slowly found me. put hyper-man...


## **Feature Extraction**

### **Extracting keywords using BERT and RAKE**

In [ ]:
bertmodel = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [ ]:
def key(text):
  r = Rake()
  temp=text.split(' ')
  a=[]
  r.extract_keywords_from_sentences(temp)
  # To get keyword phrases ranked highest to lowest.
  r.get_ranked_phrases()
  rakew=[]
  # To get keyword phrases ranked highest to lowest with scores.
  a=r.get_ranked_phrases()
  t = 20
  if(len(a)<20):
    t = len(a)
  for i in range(0,t):
    rakew.append(a[i])
  
  temp=text.split(".")
  all_words = [i for i in temp]
  sent_vector=[]
  temp=text.split(".")
  t=''
  for i in temp:
    t+=i

  n_gram_range = (1, 1)
  stop_words = "english"

  # Extract candidate words/phrases
  count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([t])
  candidates = count.get_feature_names()

  doc_embedding = bertmodel.encode([t])
  candidate_embeddings = bertmodel.encode(candidates)

  top_n = 15
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

  f_keywords = list(keywords+rakew)

  return f_keywords

### **Calculating TF-IDF values for the keywords**

In [ ]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result.keys()
    
def caltfidf(total_words,text):
  total_word_length = len(total_words)
  total_sentences = tokenize.sent_tokenize(text)
  total_sent_len = len(total_sentences)

  tf_score = {}
  for each_word in total_words:
      each_word = each_word.replace('.','')
      if each_word not in stop_words:
          if each_word in tf_score:
              tf_score[each_word] += 1
          else:
              tf_score[each_word] = 1

  # Dividing by total_word_length for each dictionary element
  tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())

  idf_score = {}
  for each_word in total_words:
      each_word = each_word.replace('.','')
      if each_word not in stop_words:
          if each_word in idf_score:
              idf_score[each_word] = check_sent(each_word, total_sentences)
          else:
              idf_score[each_word] = 1

  # Performing a log and divide
  idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())


  tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

  t = 25
  if(len(tf_idf_score)<25):
    t = len(tf_idf_score)

  tfidfw=list(get_top_n(tf_idf_score, t))
  return tfidfw

### **POS Tagging the resultant keywords**

In [ ]:
def features(text):
  f_keywords = key(text)
  tfidfw = caltfidf(f_keywords,text)
  tfidfw = " ".join(tfidfw)

  stop_words = set(stopwords.words('english'))

  tokenized = sent_tokenize(tfidfw)

  for i in tokenized:
    wordsList = nltk.word_tokenize(i)

    wordsList = [w for w in wordsList if not w in stop_words]

    tagged = nltk.pos_tag(wordsList)


  
  for i in range(0,len(tagged)):
    if(tagged[i][1]=='JJ' or tagged[i][1]=='NN' or tagged[i][1]=='VBP' or tagged[i][1]=='VBG'):
      print(tagged[i][0])

### **Removing Stopwords from the given text**

In [ ]:
def findFeatures(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    finalText = ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

    features(finalText)

### **Feature Extraction for the reviews from the dataset**

In [ ]:
test = 6
text = ' '.join([line for line in df_new['review'][test]])
features(text)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


sleep
day
expensive
drowsiness
used
side
return
study
life
reluctant
sex
work
medication
want


### **Feature Extraction for user input text**

In [ ]:
text = "The surface level is the actual realization of words as they appear in the final form. The lexical level corresponds to the combination of roots and affixes that are chained together with boundary markers."
findFeatures(text)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


combination
actual
appear
affixes
level
form
realization
final
surface
lexical
boundary


## **Text Summarization**

In [ ]:
def Summarizer(test):
  df_new["review"][test]
  sentence = sent_tokenize(str(df_new["review"][test]))
  temp=df_new["rev"][test].split(".")
  all_words = [i for i in temp]
  #model = Word2Vec([all_words], min_count=1,size= 300)
  sent_vector=[]
  temp=df_new["rev"][test].split(".")
  for i in temp:
      plus=0
      for j in i.split("."):
          #plus+=model.wv[j]
          plus+= embedder.encode(j)
      plus = plus/len(i.split("."))
      sent_vector.append(plus)
  n_clusters = 3

  kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
  y_kmeans = kmeans.fit_predict(sent_vector)
  my_list=[]
  for i in range(n_clusters):
      my_dict={}
    
      for j in range(len(y_kmeans)):
        
          if y_kmeans[j]==i:
              my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sent_vector[j])
      min_distance = min(my_dict.values())
      my_list.append(min(my_dict, key=my_dict.get))

  #print("K-Means Clusters:")
  km = []
  # print(len(sentence))
  # print(my_list)                          
  for i in sorted(my_list):
    if(i < len(sentence)):
      km.append(sentence[i])
      # print(sentence[i-1])


  model = AgglomerativeClustering(n_clusters=3)
  # fit model and predict clusters
  yhat = model.fit_predict(sent_vector)
  # retrieve unique clusters
  clusters = unique(yhat)


  my_list=[]
  for i in range(n_clusters):
      my_dict={}
    
      for j in range(len(yhat)):
        
          if yhat[j]==i:
              my_dict[j] =  distance.euclidean(clusters[i],sent_vector[j])
      min_distance = min(my_dict.values())
      my_list.append(min(my_dict, key=my_dict.get))

  #print("Agglomerative Clusters:")
  ag = []
  #print(my_list)                          
  for i in sorted(my_list):
    if(i < len(sentence)):
      ag.append(sentence[i])
      #print(sentence[i])

  final_summary = list(set(km+ag))
  print("Summary:")
  for i in final_summary:
    print(i, end=' ')
  print("\n")

In [ ]:
medicine_name = "Contrave"
ind = df_new[df_new['drugName']==medicine_name].index.values
test = ind[0]
print("\nMedicine Name: ", medicine_name)
print("\nReview: ", ' '.join(df_new['review'][test]), "\n")
Summarizer(test)
print("\nKeywords: ")
findFeatures(df_new['review'][test])


Medicine Name:  Contrave

Review:  Contrave combines drugs that were used for alcohol smoking and opioid cessation. People lose weight on it because it also helps control over-eating. I have no doubt that most obesity is caused from sugar/carb addiction which is just as powerful as any drug. I have been taking it for five days and the good news is it seems to go to work immediately. I feel hungry before I want food now. I really don't care to eat; it's just to fill my stomach. Since I have only been on it a few days I don't know if I've lost weight (I don't have a scale) but my clothes do feel a little looser so maybe a pound or two. I'm hoping that after a few months on this medication I will develop healthier habits that I can continue without the aid of Contrave. It worked immediately. I made healthier choices. I didn't have any junk food cravings not even think about them at all. Within 2wks I lost a good 16-18lbs. I loved the new me.. The healthier looking me.. Been on Contrave f

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


addictive
hypertension
drunk
diabetic
migraine
medicationstarting
cholesterol
obesity
caffeine
pillstarted
outpatient
thanksgiving
lol
eh
nahhh
speedy
quot
wise
loss
dose
drastic
fix
quot
sugar
quot
update
sw
hi
